In [1]:
!pip install streamlit transformers sentencepiece torch pyngrok > /dev/null



In [2]:
from pyngrok import ngrok

# 🔑 Replace with your own token
NGROK_AUTH_TOKEN = "333DOnc5UTdGeA9xhoVfzv0P75I_7LaXthSSmqK7ypvqyHpTU"

ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [3]:
import re
import streamlit as st
import pandas as pd
import altair as alt
from transformers import MarianMTModel, MarianTokenizer
import json
import hashlib
import os
from collections import Counter
from datetime import datetime
import numpy as np # Import for language metrics

# --- Constants for User Management ---
USERS_FILE = "users.json"
METRICS_FILE = "metrics.json" # <--- NEW CONSTANT FOR PERSISTENCE
ADMIN_USERNAME = "admin"
ADMIN_PASSWORD_HASH = hashlib.sha256("healthbotadmin123".encode()).hexdigest() # Hardcoded Admin Pass: healthbotadmin123
# --- End of Constants ---

# --- Utility Functions for Persistence ---
def load_users():
    """Loads user data from the JSON file."""
    if os.path.exists(USERS_FILE):
        try:
            with open(USERS_FILE, 'r') as f:
                return json.load(f)
        except json.JSONDecodeError:
            st.error("Error loading users.json. File might be corrupted.")
            return {}
    return {}

def save_users(users_data):
    """Saves user data to the JSON file."""
    with open(USERS_FILE, 'w') as f:
        json.dump(users_data, f, indent=4)

def hash_password(password):
    """Hashes the password using SHA256."""
    return hashlib.sha256(password.encode()).hexdigest()

def load_metrics(): # <--- NEW FUNCTION
    """Loads metrics data from the JSON file."""
    if os.path.exists(METRICS_FILE):
        try:
            with open(METRICS_FILE, 'r') as f:
                return json.load(f)
        except json.JSONDecodeError:
            # Handle empty or corrupted file by returning a safe default
            return []
    return []

def save_metrics(metrics_data): # <--- NEW FUNCTION
    """Saves metrics data to the JSON file."""
    with open(METRICS_FILE, 'w') as f:
        json.dump(metrics_data, f, indent=4)

def log_query(username, user_input, detected_entities, bot_response, is_hindi): # MODIFIED: Added is_hindi
    """Logs a successful query to the metrics file."""
    metrics = load_metrics()

    # Generate a simple unique ID for this query (using timestamp for simplicity)
    query_id = datetime.now().isoformat()

    metrics.append({
        "query_id": query_id,
        "username": username,
        "timestamp": query_id,
        "input": user_input,
        "entities": detected_entities,
        "response": bot_response,
        "feedback": None, # Initial state
        "language": "Hindi" if is_hindi else "English" # Log the language
    })
    save_metrics(metrics)
    st.session_state.persistent_metrics = metrics # Update session state immediately

def log_feedback(query_timestamp, feedback_emoji): # MODIFIED: Uses query_timestamp as ID
    """Updates the feedback status for a logged query."""
    metrics = load_metrics()

    # Find the matching query by timestamp (used as a proxy ID)
    for i in range(len(metrics) - 1, -1, -1):
        if metrics[i].get("timestamp") == query_timestamp:
            metrics[i]['feedback'] = feedback_emoji
            save_metrics(metrics)
            st.session_state.persistent_metrics = metrics # Update session state immediately
            return
# --- End of Utility Functions ---

# --- Knowledge Base Management (CRUD) ---
knowledge_base = {
    "fever": {"first_aid": "Rest, drink fluids, and monitor your temperature.",
              "prevention": "Wash hands often and avoid contact with sick people."},
    "headache": {"first_aid": "Rest in a quiet place and stay hydrated.",
                  "prevention": "Get enough sleep and reduce screen exposure."},
    "cough": {"first_aid": "Drink warm water and rest your voice.",
              "prevention": "Avoid dust and cold drinks."},
    "cold": {"first_aid": "Stay warm, drink ginger tea, and rest.",
              "prevention": "Keep hygiene and avoid cold exposure."},
    "stomach pain": {"first_aid": "Rest, drink water, and eat light food.",
                      "prevention": "Avoid spicy foods and overeating."},
    "sore throat": {"first_aid": "Gargle with warm salt water and drink warm fluids.",
                      "prevention": "Avoid shouting and stay hydrated."},
    "diarrhea": {"first_aid": "Drink ORS and stay hydrated.",
                  "prevention": "Eat clean food and wash hands."},
    "vomiting": {"first_aid": "Drink small sips of water or ORS. Rest.",
                  "prevention": "Avoid stale or spicy food."},
    "burn": {"first_aid": "Cool the burn with running water for 10 mins.",
              "prevention": "Keep hot objects away and use gloves."},
    "cut": {"first_aid": "Clean the wound and cover with a sterile bandage.",
              "prevention": "Be cautious with sharp tools."},
    "anxiety": {"first_aid": "Practice deep breathing or meditation.",
                  "prevention": "Maintain a healthy routine."},
    "fatigue": {"first_aid": "Get enough rest and eat nutritious food.",
                  "prevention": "Maintain a regular sleep schedule."},
    "flu": {"first_aid": "Rest, drink warm fluids, and monitor symptoms.",
              "prevention": "Get vaccinated and maintain hygiene."}
}

if 'knowledge_base_state' not in st.session_state:
    st.session_state.knowledge_base_state = knowledge_base

def update_knowledge_base_add(symptom, first_aid, prevention):
    st.session_state.knowledge_base_state[symptom.lower()] = {
        "first_aid": first_aid,
        "prevention": prevention
    }

def update_knowledge_base_delete(symptom):
    if symptom.lower() in st.session_state.knowledge_base_state:
        del st.session_state.knowledge_base_state[symptom.lower()]
# --- End of Knowledge Base Management ---

@st.cache_resource
def load_translation_models():
    # Model loading logic remains the same
    en_hi_model_name = "Helsinki-NLP/opus-mt-en-hi"
    hi_en_model_name = "Helsinki-NLP/opus-mt-hi-en"
    en_hi_tok = MarianTokenizer.from_pretrained(en_hi_model_name)
    en_hi_model = MarianMTModel.from_pretrained(en_hi_model_name)
    hi_en_tok = MarianTokenizer.from_pretrained(hi_en_model_name)
    hi_en_model = MarianMTModel.from_pretrained(hi_en_model_name)
    return en_hi_tok, en_hi_model, hi_en_tok, hi_en_model

# Load models and assign them to global names (with error handling)
try:
    en_hi_tok, en_hi_model, hi_en_tok, hi_en_model = load_translation_models()
except Exception as e:
    st.info(f"Translation models are not loaded in this environment. Translation features may not work: {e}")
    # Define dummy translate function if models fail
    def translate(text, tok=None, model=None):
        return text

def translate(text, tok, model):
    toks = tok(text, return_tensors="pt", padding=True)
    out = model.generate(**toks)
    return tok.decode(out[0], skip_special_tokens=True)


def extract_entities(text_en):
    entities = []
    kb = st.session_state.knowledge_base_state
    for key in kb.keys():
        pattern = r"\b" + re.escape(key) + r"\b"
        if re.search(pattern, text_en.lower()):
            entities.append(key)
    return entities


def generate_response(entities, user_profile, lang="en"):
    # ... (generate_response logic remains the same) ...
    prefix_en = "According to **WHO guidelines**, here is some general advice:\n"
    prefix_hi = "WHO के दिशानिर्देशों के अनुसार, यहां कुछ सामान्य सलाह दी गई है:\n"

    advice_points_en = []
    kb = st.session_state.knowledge_base_state # Use the current KB

    # --- Profile-based Context (for bullet point) ---
    age_context = ""
    if user_profile:
        if user_profile.get("age"):
            age = user_profile["age"]
            if age < 12:
                age_context += "* **Child Alert:** Since this is a child, please be extra careful and consult a pediatrician.\n"
            elif age > 60:
                age_context += "* **Elderly Alert:** Elderly individuals should monitor symptoms closely and consult a doctor immediately.\n"

        if 'bmi_status' in user_profile:
            bmi_status = user_profile['bmi_status']
            age_context += f"* **BMI Status:** Your current BMI status is **{bmi_status}**.\n"

    # --- Core Advice (Bullet Points) ---
    if not entities:
        base = "* Please provide more details about your symptoms."
    else:
        for ent in entities:
            if ent in kb:
                advice_points_en.append(f"* **{ent.capitalize()}**: First Aid: {kb[ent]['first_aid']}. Prevention: {kb[ent]['prevention']}.")

        base = "\n".join(advice_points_en)

    # Combine Core Advice and Context
    base = base + "\n" + age_context

    # --- Dynamic Disclaimer (End of response) ---
    if entities and (user_profile.get("age", 0) < 12 or user_profile.get("age", 0) > 60):
        disclaimer_en = "\n\n🚨 **URGENT DISCLAIMER:** Given your profile (age), please consult a doctor immediately. This advice is not a diagnosis."
        disclaimer_hi = "\n\n🚨 **अत्यावश्यक अस्वीकरण:** आपकी प्रोफ़ाइल (उम्र) को देखते हुए, तुरंत डॉक्टर से सलाह लें। यह सलाह निदान नहीं है।"
    else:
        disclaimer_en = "\n\n⚠️ **DISCLAIMER:** This is general wellness advice, not a medical diagnosis. Please consult a doctor."
        disclaimer_hi = "\n\n⚠️ **अस्वीकरण:** यह केवल सामान्य जानकारी है, कृपया डॉक्टर से परामर्श लें।"

    # Construct the full response in English
    response_en = (prefix_en if entities else "") + base + (disclaimer_en if entities else "")

    # Translate if necessary
    if lang == "hi":
        response_final = translate(response_en, en_hi_tok, en_hi_model)
    else:
        response_final = response_en

    return response_final


# -----------------------------
# 🧠 Chatbot Logic
# -----------------------------
def health_chatbot(user_input, user_profile):
    is_hindi = bool(re.search(r'[^\x00-\x7F]', user_input))
    if is_hindi:
        text_en = translate(user_input, hi_en_tok, hi_en_model)
    else:
        text_en = user_input

    entities = extract_entities(text_en)

    response_final = generate_response(entities, user_profile, "hi" if is_hindi else "en")

    # Generate a unique timestamp ID for this query log
    current_time_str = datetime.now().isoformat()

    # Log the query to persistent storage
    log_query(
        username=st.session_state.username,
        user_input=user_input,
        detected_entities=entities,
        bot_response=response_final,
        is_hindi=is_hindi # Pass language flag for logging
    )

    return response_final, current_time_str # Return the response and the log ID

# ... (calculate_bmi logic remains the same) ...
def calculate_bmi(height_cm, weight_kg):
    if height_cm > 0 and weight_kg > 0:
        height_m = height_cm / 100
        bmi = weight_kg / (height_m ** 2)

        if bmi < 18.5:
            status = "Underweight 😟"
            progress = 0.2
            color = "red"
        elif 18.5 <= bmi < 25:
            status = "Healthy weight! 😊"
            progress = 0.5
            color = "green"
        elif 25 <= bmi < 30:
            status = "Overweight 😕"
            progress = 0.8
            color = "orange"
        else:
            status = "Obese ❗"
            progress = 1.0
            color = "red"

        return f"{bmi:.2f}", status, progress, color
    return "N/A", "N/A", 0, "gray"

# -----------------------------
# 🎨 Streamlit UI for Main App (Chatbot)
# -----------------------------
def main_app():
    st.title("🩺 Multilingual Health Chatbot")
    st.caption("Get quick health guidance in English, Hindi, or Hinglish. (Not a substitute for professional medical advice)")
    st.markdown(f"**Welcome, {st.session_state.username}!**")

    # --- Sidebar for user health dashboard ---
    with st.sidebar:
        st.header("👤 Your Health Dashboard")

        if st.button("🚪 Logout"):
            st.session_state.logged_in = False
            st.session_state.username = None
            st.session_state.page = "login"
            st.rerun()

        st.markdown("---")

        # Input widgets (unchanged)
        height = st.number_input("Height (cm)", min_value=50, max_value=250, value=st.session_state.user_profile.get("height", 170), key="profile_height")
        weight = st.number_input("Weight (kg)", min_value=10, max_value=200, value=st.session_state.user_profile.get("weight", 70), key="profile_weight")
        age = st.number_input("Age", min_value=1, max_value=120, value=st.session_state.user_profile.get("age", 25), key="profile_age")
        gender = st.selectbox("Gender", ["Male", "Female", "Other"], index=["Male", "Female", "Other"].index(st.session_state.user_profile.get("gender", "Male")), key="profile_gender")
        blood_group = st.selectbox("Blood Group", ["A+", "A-", "B+", "B-", "O+", "O-", "AB+", "AB-"], index=["A+", "A-", "B+", "B-", "O+", "O-", "AB+", "AB-"].index(st.session_state.user_profile.get("blood_group", "O+")), key="profile_blood")

        bmi_value, bmi_status, bmi_progress, bmi_color = calculate_bmi(height, weight)

        # Update and Save profile
        st.session_state.user_profile = {
            "age": age, "gender": gender, "blood_group": blood_group,
            "height": height, "weight": weight, "bmi_status": bmi_status
        }
        users = load_users()
        users[st.session_state.username]['profile'] = st.session_state.user_profile
        save_users(users)

        # Display BMI (unchanged)
        st.markdown("---")
        st.subheader("Your Health Metrics")
        col_bmi, col_status = st.columns(2)
        with col_bmi: st.metric(label="BMI Value", value=bmi_value)
        with col_status:
            st.markdown(f"**Status:**")
            st.markdown(f"**<span style='color:{bmi_color}'>{bmi_status}</span>**", unsafe_allow_html=True)
        st.progress(bmi_progress, text="BMI Range Progress")
        st.markdown("---")

    # --- Chat input and flow logic (Modified to read from persistent_metrics) ---
    st.divider()
    st.subheader("💬 Chat with Health Assistant")

    # Filter persistent metrics for the current user's chat history
    user_chat_history = [
        msg for msg in st.session_state.persistent_metrics
        if msg.get("username") == st.session_state.username
    ]

    temp_chat_display = []
    for msg in user_chat_history:
        # User message
        temp_chat_display.append({"role": "user", "content": msg['input']})
        # Bot message, including persistent ID (timestamp)
        temp_chat_display.append({
            "role": "bot",
            "content": msg['response'],
            "feedback": msg['feedback'],
            "timestamp": msg['timestamp'] # The unique identifier
        })


    for msg in temp_chat_display:
        if msg["role"] == "user":
            st.markdown(f"🧑‍💬 **You:** {msg['content']}")
        else:
            st.markdown(f"🤖 **Bot:**")
            st.markdown(msg['content'])

            # Add Feedback Controls for Bot's Response
            col_feedback_1, col_feedback_2, col_feedback_3 = st.columns([0.1, 0.1, 0.8])
            query_timestamp = msg['timestamp']
            current_feedback = msg['feedback']

            # Use a lambda function to handle feedback logging and rerunning
            def set_feedback_callback(ts, emoji):
                log_feedback(ts, emoji)
                st.rerun()

            with col_feedback_1:
                is_like = current_feedback == "👍"
                if st.button("👍", key=f"feedback_{query_timestamp}_like", disabled=is_like, on_click=set_feedback_callback, args=(query_timestamp, "👍")):
                    pass

            with col_feedback_2:
                is_dislike = current_feedback == "👎"
                if st.button("👎", key=f"feedback_{query_timestamp}_dislike", disabled=is_dislike, on_click=set_feedback_callback, args=(query_timestamp, "👎")):
                    pass

            with col_feedback_3:
                if current_feedback:
                    st.markdown(f"**Feedback received:** {current_feedback}")
            st.markdown("---")

    def process_input(key):
        user_input = st.session_state[key].strip()
        if user_input:
            health_chatbot(user_input, st.session_state.user_profile)
            st.session_state[key] = ""
            st.rerun() # Rerun to display the new persistent data

    st.text_input(
        "Type your query here...",
        key="current_input",
        on_change=process_input,
        args=("current_input",),
        placeholder="e.g., I have a fever and fatigue"
    )

    if st.button("🧹 Clear Chat (Local Only)"):
        st.warning("Chat is persistent. This button does nothing in this demo version.")

    # --- Symptom Trend Visualization (Modified to read from persistent_metrics) ---
    st.markdown("---")
    st.subheader("📊 Your Symptom Trends")

    user_symptoms = []
    for msg in user_chat_history:
        user_symptoms.extend(msg.get("entities", []))

    if user_symptoms:
        symptom_counts = pd.Series(user_symptoms).value_counts().reset_index()
        symptom_counts.columns = ['Symptom', 'Count']
        symptom_counts['Count'] = pd.to_numeric(symptom_counts['Count'], errors='coerce').fillna(0).astype(int)
        top_symptoms = symptom_counts.head(5)

        chart = alt.Chart(top_symptoms).mark_bar().encode(
            x=alt.X('Symptom', sort='-y'), y='Count', tooltip=['Symptom', 'Count'],
            color=alt.condition(alt.datum.Count == top_symptoms['Count'].max(), alt.value('red'), alt.value('steelblue'))
        ).properties(title='Top 5 Reported Symptoms').interactive()

        st.altair_chart(chart, use_container_width=True)
    else:
        st.info("Start chatting to see your most frequently reported symptoms here.")


    st.markdown("---")
    st.caption("⚕️ General health assistant — not a replacement for a doctor.")

# -----------------------------
# 👑 Admin Dashboard UI (Refactored with Tabs)
# -----------------------------
def admin_dashboard():
    st.title("👑 Admin Dashboard")
    st.markdown("Monitor system health, manage the Knowledge Base, and analyze bot performance.")

    # --- Logout Button ---
    if st.button("🚪 Logout Admin"):
        st.session_state.is_admin_logged_in = False
        st.session_state.page = "login"
        st.rerun()
    st.markdown("---")

    # Load persistent data for the dashboard metrics
    metrics_data = st.session_state.persistent_metrics
    users = load_users()
    total_users = len(users)
    total_queries = len(metrics_data)

    # --- Refactored Tabs ---
    tab_dash, tab_kb, tab_analytics, tab_users = st.tabs(["📊 Dashboard Summary", "📚 Knowledge Base", "📈 Analytics & Gaps", "👤 User Management"])

    # =======================================================
    # TAB 1: DASHBOARD SUMMARY
    # =======================================================
    with tab_dash:
        st.subheader("System Health and Performance Monitoring")

        col1, col2, col3, col4 = st.columns(4)

        col1.metric("👥 Total Registered Users", total_users)
        col2.metric("💬 Total Queries Handled", total_queries)
        col3.metric("🧠 Health Topics (KB Size)", len(st.session_state.knowledge_base_state))
        col4.metric("➕ New Users Today", 0, "No tracking (demo)")

        st.markdown("---")

        st.subheader("Performance & Feedback")

        # C: Symptom Analysis (Top 5 Reported Symptoms)
        all_symptoms = [symptom for msg in metrics_data for symptom in msg.get("entities", [])]
        symptom_counts = Counter(all_symptoms)
        symptom_data = pd.DataFrame(symptom_counts.items(), columns=['Symptom', 'Count'])

        if not symptom_data.empty:
            symptom_data['Count'] = pd.to_numeric(symptom_data['Count'], errors='coerce').fillna(0).astype(int)
            symptom_data = symptom_data.nlargest(5, 'Count')

        # D: Feedback Review (PIE CHART ONLY)
        likes = sum(1 for msg in metrics_data if msg.get('feedback') == '👍')
        dislikes = sum(1 for msg in metrics_data if msg.get('feedback') == '👎')
        total_feedback = likes + dislikes

        col_symp, col_feed = st.columns([1, 1])

        with col_symp:
            st.markdown("#### Top 5 Reported Symptoms (All Users)")
            if not symptom_data.empty:
                chart = alt.Chart(symptom_data).mark_bar().encode(
                    x=alt.X('Symptom', sort='-y'), y='Count', tooltip=['Symptom', 'Count'],
                    color=alt.value('darkred')
                ).properties(height=300)
                st.altair_chart(chart, use_container_width=True)
            else:
                st.info("No symptom data yet.")

        with col_feed:
            st.markdown("#### Bot Feedback Summary")
            if total_feedback > 0:
                feedback_data = pd.DataFrame({
                    'Category': ['👍 Likes', '👎 Dislikes'],
                    'Count': [likes, dislikes]
                })

                # PIE CHART ONLY (requested fix)
                pie_chart = alt.Chart(feedback_data).mark_arc(outerRadius=100, innerRadius=50).encode(
                    theta=alt.Theta("Count", stack=True),
                    color=alt.Color("Category", scale=alt.Scale(domain=['👍 Likes', '👎 Dislikes'], range=['green', 'red'])),
                    order=alt.Order("Count", sort="descending"),
                    tooltip=['Category', 'Count']
                ).properties(title="Feedback Distribution", height=250)
                st.altair_chart(pie_chart, use_container_width=True)

                st.metric("Total Feedback Responses", total_feedback)
            else:
                st.info("No bot feedback recorded yet.")

        st.markdown("---")

        # 3. User Activity Trend (FIXED: Cumulative Trend)
        st.subheader("🗓️ Cumulative Query Activity Trend")
        if total_queries > 0:
            query_timestamps = [msg['timestamp'] for msg in metrics_data]
            trend_data = pd.DataFrame({'Query Time': pd.to_datetime(query_timestamps)})

            # Calculate Cumulative Queries (Daily)
            trend_data['Date'] = trend_data['Query Time'].dt.date
            daily_query_count = trend_data.groupby('Date').size().reset_index(name='Daily Count')
            daily_query_count['Cumulative Queries'] = daily_query_count['Daily Count'].cumsum()
            daily_query_count['Date'] = daily_query_count['Date'].astype(str)

            trend_chart = alt.Chart(daily_query_count).mark_line(point=True).encode(
                x=alt.X('Date', title='Date of Activity'),
                y='Cumulative Queries',
                tooltip=['Date', 'Cumulative Queries', 'Daily Count']
            ).properties(
                title='Cumulative Queries Handled Over Time (Persistence Enabled)'
            ).interactive()

            st.altair_chart(trend_chart, use_container_width=True)
        else:
            st.info("No query activity to plot yet.")

    # =======================================================
    # TAB 2: KNOWLEDGE BASE MANAGEMENT
    # =======================================================
    with tab_kb:
        st.subheader("📚 Knowledge Base Management")

        kb_df = pd.DataFrame.from_dict(st.session_state.knowledge_base_state, orient='index').reset_index()
        kb_df.columns = ['Symptom', 'First Aid', 'Prevention']

        st.markdown("##### Current Knowledge Base Topics")
        st.dataframe(kb_df, height=300, use_container_width=True)

        tab_add, tab_delete = st.tabs(["➕ Add New Topic", "❌ Delete Topic"])

        with tab_add:
            with st.form("add_kb_form"):
                new_symptom = st.text_input("New Symptom/Condition (e.g., 'allergy')")
                new_first_aid = st.text_area("First Aid Advice")
                new_prevention = st.text_area("Prevention Tip")
                submitted = st.form_submit_button("Add to Knowledge Base")

                if submitted and new_symptom and new_first_aid and new_prevention:
                    update_knowledge_base_add(new_symptom, new_first_aid, new_prevention)
                    st.success(f"Successfully added **{new_symptom}** to the knowledge base!")
                    st.rerun()
                elif submitted:
                    st.error("Please fill in all fields.")

        with tab_delete:
            symptom_to_delete = st.selectbox("Select Symptom to Delete", options=sorted(st.session_state.knowledge_base_state.keys()))
            if st.button(f"Delete '{symptom_to_delete}'"):
                update_knowledge_base_delete(symptom_to_delete)
                st.warning(f"Successfully deleted **{symptom_to_delete}**.")
                st.rerun()

    # =======================================================
    # TAB 3: ANALYTICS & GAPS (New Features 1 & 3)
    # =======================================================
    with tab_analytics:
        st.subheader("📈 Content & Language Analytics")
        st.markdown("---")

        # --- Feature: Language Usage Pie Chart ---
        st.markdown("##### 🌍 Language Usage Distribution")
        lang_counts = Counter(msg.get("language", "Unknown") for msg in metrics_data)
        lang_data = pd.DataFrame(lang_counts.items(), columns=['Language', 'Count'])

        if not lang_data.empty:
            lang_chart = alt.Chart(lang_data).mark_arc(outerRadius=100).encode(
                theta=alt.Theta("Count", stack=True),
                color=alt.Color("Language", scale=alt.Scale(domain=['English', 'Hindi', 'Unknown'], range=['#3498db', '#e74c3c', '#95a5a6'])),
                tooltip=['Language', 'Count']
            ).properties(title="Query Language Distribution")
            st.altair_chart(lang_chart, use_container_width=True)
        else:
            st.info("No language data recorded yet.")

        st.markdown("---")

        # --- Feature: Top Unanswered Queries (Content Gap) ---
        st.markdown("##### 🔍 Top Unanswered Queries (Content Gap Analysis)")
        unanswered_queries = [
            msg["input"] for msg in metrics_data
            if not msg.get("entities")
        ]

        if unanswered_queries:
            gap_counts = pd.Series(unanswered_queries).value_counts().reset_index()
            gap_counts.columns = ['Query Text', 'Count']

            st.markdown("These are the most frequent queries that did **NOT** match any symptom in the KB.")
            st.dataframe(gap_counts.head(5), hide_index=True)
        else:
            st.info("Every query has matched at least one symptom! Good job, or start typing more diverse queries.")

        st.markdown("---")

        # --- Feature: Top Disliked Responses (Bot Quality) ---
        st.markdown("##### 👎 Top Disliked Responses (Bot Quality)")
        disliked_records = [
            msg for msg in metrics_data
            if msg.get("feedback") == '👎'
        ]

        if disliked_records:
            # Sort by timestamp to get the most recent dislikes first
            disliked_records.sort(key=lambda x: x['timestamp'], reverse=True)
            st.markdown("Review the most recent queries that received a **👎 Dislike** feedback:")

            for record in disliked_records[:5]:
                with st.expander(f"👎 **{record['timestamp'][:16].replace('T', ' ')}** by **{record['username']}**"):
                    st.markdown(f"**User Query:** {record['input']}")
                    st.markdown(f"**Bot Response:** {record['response']}")
                    st.markdown(f"**Detected Symptoms:** {', '.join(record['entities'])}")
        else:
            st.info("No dislike feedback recorded yet.")


    # =======================================================
    # TAB 4: USER MANAGEMENT (New Feature 2)
    # =======================================================
    with tab_users:
        st.subheader("👤 User Activity & Management")

        # --- Feature: User Query Lookup ---
        available_users = sorted(users.keys())
        selected_user = st.selectbox("Select User to Inspect", options=available_users)

        if selected_user:
            st.markdown("---")
            st.markdown(f"#### Profile and Activity for **{selected_user}**")

            # User Profile
            user_data = users[selected_user]
            profile = user_data.get('profile', {})

            col_info, col_metrics = st.columns(2)

            with col_info:
                st.markdown("**Basic Information**")
                st.json({k: v for k, v in profile.items() if k not in ['bmi_status']})

            # User Metrics
            user_metrics = [msg for msg in metrics_data if msg.get("username") == selected_user]
            total_user_queries = len(user_metrics)
            user_likes = sum(1 for msg in user_metrics if msg.get('feedback') == '👍')

            with col_metrics:
                st.markdown("**Session Metrics**")
                st.metric("Total Queries", total_user_queries)
                st.metric("Positive Feedback (👍)", user_likes)

            st.markdown("---")

            st.markdown("##### Last 5 Queries")
            if user_metrics:
                # Sort by timestamp (most recent first)
                user_metrics.sort(key=lambda x: x['timestamp'], reverse=True)

                for record in user_metrics[:5]:
                    feedback_status = record.get('feedback', 'No feedback')
                    st.code(f"Time: {record['timestamp'][11:19]} | Feedback: {feedback_status}")
                    st.markdown(f"**Input:** {record['input']}")
                    st.markdown(f"**Response Snippet:** {record['response'][:80]}...")
                    st.markdown("---")
            else:
                st.info(f"No queries found for {selected_user}.")


# -----------------------------
# 🔒 Authentication Pages (Unchanged)
# -----------------------------
def admin_login_page():
    st.title("Admin Login 🔑")

    with st.form("admin_login_form"):
        username = st.text_input("Admin Username")
        password = st.text_input("Admin Password", type="password")
        submitted = st.form_submit_button("Login as Admin")

        if submitted:
            hashed_pw = hash_password(password)
            if username == ADMIN_USERNAME and hashed_pw == ADMIN_PASSWORD_HASH:
                st.session_state.is_admin_logged_in = True
                st.success("Admin login successful!")
                st.rerun()
            else:
                st.error("Invalid Admin Credentials.")

    st.markdown("---")
    if st.button("⬅️ Back to User Login"):
        st.session_state.page = "login"
        st.rerun()

def login_page():
    st.title("Welcome to the Health Chatbot")
    st.subheader("Login to Continue")

    users = load_users()

    with st.form("login_form"):
        username = st.text_input("Username")
        password = st.text_input("Password", type="password")
        submitted = st.form_submit_button("Login")

        if submitted:
            hashed_pw = hash_password(password)
            if username in users and users[username]['password'] == hashed_pw:
                st.session_state.logged_in = True
                st.session_state.username = username
                st.session_state.user_profile = users[username].get('profile', {})
                st.success(f"Welcome back, {username}!")
                st.rerun()
            else:
                st.error("Invalid Username or Password.")

    st.markdown("---")

    col_signup, col_admin = st.columns(2)
    with col_signup:
        if st.button("Don't have an account? Sign Up"):
            st.session_state.page = "signup"
            st.rerun()
    with col_admin:
        if st.button("Admin Access"):
            st.session_state.page = "admin_login"
            st.rerun()

def signup_page():
    st.title("Health Chatbot Sign Up")
    st.subheader("Create a New Account")

    users = load_users()

    with st.form("signup_form"):
        new_username = st.text_input("Choose Username")
        new_password = st.text_input("Choose Password", type="password")
        confirm_password = st.text_input("Confirm Password", type="password")
        submitted = st.form_submit_button("Sign Up")

        if submitted:
            if new_username in users:
                st.error("Username already taken. Please choose another.")
            elif new_password != confirm_password:
                st.error("Passwords do not match.")
            elif len(new_username) < 3 or len(new_password) < 6:
                st.error("Username must be at least 3 characters and password at least 6 characters.")
            else:
                hashed_pw = hash_password(new_password)
                users[new_username] = {'password': hashed_pw, 'profile': {}}
                save_users(users)
                st.success("Account created successfully! Please log in.")
                st.session_state.page = "login"
                st.rerun()

    st.markdown("---")
    if st.button("Already have an account? Log In"):
        st.session_state.page = "login"
        st.rerun()

# -----------------------------
# 🚀 Main Application Flow Control
# -----------------------------
def app():
    st.set_page_config(page_title="🩺 Enhanced Health Chatbot", layout="wide")

    # --- Global Initialization Block (FIXED for Persistence) ---
    if "logged_in" not in st.session_state: st.session_state.logged_in = False
    if "is_admin_logged_in" not in st.session_state: st.session_state.is_admin_logged_in = False
    if "username" not in st.session_state: st.session_state.username = None
    if "page" not in st.session_state: st.session_state.page = "login"

    # FIX: Load the persistent metrics data from the file into session state on every run
    if "persistent_metrics" not in st.session_state:
        st.session_state.persistent_metrics = load_metrics()

    if "user_profile" not in st.session_state:
        st.session_state.user_profile = {}
    # --- END Global Initialization Block ---


    if st.session_state.is_admin_logged_in:
        admin_dashboard()
    elif st.session_state.logged_in:
        main_app()
    else:
        if st.session_state.page == "login":
            login_page()
        elif st.session_state.page == "signup":
            signup_page()
        elif st.session_state.page == "admin_login":
            admin_login_page()

if __name__ == '__main__':
    app()

Writing Milestone3_Auth.py


In [5]:
from pyngrok import ngrok, conf
# disconnect all tunnels
for t in ngrok.get_tunnels():
    ngrok.disconnect(t.public_url)
print("✅ All previous ngrok tunnels closed.")

from pyngrok import ngrok
import os, threading, time

def start_streamlit():
    os.system("streamlit run Milestone3_Auth.py --server.port 8501 --server.headless true")

threading.Thread(target=start_streamlit, daemon=True).start()
time.sleep(8)
print("🌐 Public URL:", ngrok.connect(8501).public_url)

✅ All previous ngrok tunnels closed.
🌐 Public URL: https://8125f08fca55.ngrok-free.app
